# Capitulo 6 - Padrões de projeto com funções de primeira classe

## Strategy clássico

### Exemplo de implementação da classe Order com estratégias de desconto intercambiáveis

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity

class Order:
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion
  
  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
      
      return self.__total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion.discount(self)

    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    
    return fmt.format(self.total(), self.due())

class Promotion (ABC):

  @abstractmethod
  def discount(self, order):
    """Devolve o desconto como um valor positivo em dólares"""

class FidelityPromo(Promotion):
  """5% de desconto para clientes com mil ou mais pontos no programa de fidelidade"""
  def discount(self, order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
  """10% de desconto para cada LineItem com 20 ou mais unidades"""
  def discount(self, order):
    discount = 0
  
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * .1

        return discount

class LargeOrderPromo(Promotion):
  """7% de desconto para pedidos com 10 ou mais itens diferentes"""
  def discount(self, order):
    distinct_items = {item.product for item in order.cart}

    if len(distinct_items) >= 10:
      return order.total() * .07
    
    return 0

### Exemplo de uso da classe Order com a aplicação de diferentes promoções

In [2]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [
  LineItem('banana', 4, .5),
  LineItem('apple', 10, 1.5),
  LineItem('watermelon', 5, 5.0)
]

banana_cart = [
  LineItem('banana', 30, .5),
  LineItem('apple', 10, 1.5)
]

long_order = [
  LineItem(str(item_code), 1, 1.0)
  for item_code in range(10)
]

Order(joe, cart, FidelityPromo())
Order(ann, cart, FidelityPromo())
Order(joe, banana_cart, BulkItemPromo())
Order(joe, long_order, LargeOrderPromo())
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

## Strategy orientado a função

### Exemplo de implementação da classe Order com estratégias de desconto implementadas como funções

In [3]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
  def __init__ (self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price
  
  def total(self):
    return self.price * self.quantity

class Order:
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion
  
  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion(self)
    
    return self.total() - discount
  
  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    
    return fmt.format(self.total(), self.due())

def fidelity_promo(order):
    """5% de desconto para clientes com mil ou mais pontos no programa de fidelidade"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
  """10% de desconto para cada LineItem com 20 ou mais unidades"""
  discount = 0
  
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * .1
  
  return discount

def large_order_promo(order):
  """7% de desconto para pedidos com 10 ou mais itens diferentes"""
  distinct_items = {item.product for item in order.cart}
  
  if len(distinct_items) >= 10:
    return order.total() * .07

  return 0

### Exemplo de uso da classe Order com a aplicação de diferentes promoções

In [4]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [
  LineItem('banana', 4, .5),
  LineItem('apple', 10, 1.5),
  LineItem('watermelon', 5, 5.0)
]
banana_cart = [
  LineItem('banana', 30, .5),
  LineItem('apple', 10, 1.5)
]
long_order = [
  LineItem(str(item_code), 1, 1.0)
  for item_code in range(10)
]

Order(joe, cart, fidelity_promo)
Order(ann, cart, fidelity_promo)
Order(joe, banana_cart, bulk_item_promo)
Order(joe, long_order, large_order_promo)
Order (joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

### Escolhendo a melhor estratégia: abordagem simples

In [5]:
# best_promo encontra o desconto máximo fazendo uma iteração por uma lista de funções

promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
  """Seleciona o melhor desconto disponível"""
  return max(promo(order) for promo in promos)

In [6]:
# A função best_promo aplica todos os descontos e retorna o maior
Order(joe, long_order, best_promo)
Order(joe, banana_cart, best_promo)
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

### Encontrando estratégias em um módulo

In [7]:
# A lista promos é criada por introspecção do namespace global do módulo

promos = [
  globals()[name] 
  for name in globals()
  if name.endswith('_promo') and name != 'best_promo'
]

def best_promo(order):
  """Seleciona o melhor desconto disponível"""
  return max(promo(order) for promo in promos)

In [31]:
# A lista promos é criada por introspecção em um novo módulo promotions

import inspect

promos = [
  name
  for name, func in inspect.getmembers(promotions)
]

def best_promo(order):
  """Seleciona o melhor desconto disponível"""
  return max(promo(order) for promo in promos)

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'promotions']


## Command

In [ ]:
# Cada instância de MacroCommand tem uma lista interna de comandos

class MacroCommand:
  """Um comando que executa uma lista de comandos"""
  def __init__ (self, commands):
    self.commands = list(commands)
  
  def __call__(self):
    for command in self.commands:
      command()